This simulates a stock's price using Geometric Brownian Motion, and then prices an Asian and Barrier option using Monte Carlo simulations with varying path lengths.

**Asian Options**

Asian options are a type of exotic option where the payoff depends on the average price of the underlying asset over a certain period of time, rather than its price at a single point in time. This averaging feature makes Asian options less susceptible to market manipulation and reduces volatility. There are primarily two types of Asian options: the average price option, where the payoff depends on the average price of the underlying asset, and the average strike option, where the strike price is the average price of the underlying asset over the term of the option.

The payoff of an average price Asian call option can be represented as follows:

$$
\text{Payoff} = \text{max}\left(0, \frac{1}{T}\sum\nolimits_{t=1}^{T}S_{t} - K\right)
$$

Where $S_t$ is the price of the underlying asset at time $t$, $K$ is the strike price, and $T$ is the total period over which the average is calculated.

**Barrier Options**

Barrier options are exotic options that become activated or extinguished when the price of the underlying asset crosses a certain barrier level. These options can be classified into two main categories: knock-in options, which come into existence when the barrier is breached, and knock-out options, which are extinguished when the barrier is breached. The barrier can be set either above or below the current price of the underlying asset, leading to further subdivisions like up-and-in, up-and-out, down-and-in, and down-and-out options.

The payoff of a simple up-and-out call barrier option can be represented as:

$$
\text{Payoff} = 
\begin{cases} 
0 & \text{if }{S_{T} \ge B} \text{ at any time before expiry}\\
max(S_{T} - K,0) & \text{otherwise}
\end{cases}
$$

Where $S_T$ is the price of the underlying asset at maturity, $K$ is the strike price, and $B$ is the barrier level.

**Pricing Using Monte Carlo Simulations**

Monte Carlo simulations can be used to price both Asian and barrier options by simulating a large number of paths for the underlying asset's price movement, calculating the payoff for each path, and then averaging these payoffs. The steps for pricing these options using Monte Carlo simulations typically include:

Simulating Price Paths: Generate multiple paths for the price of the underlying asset using a stochastic process, such as the Geometric Brownian Motion (GBM), defined by: $dS_{t} = \mu S_{t} dt + \sigma S_{t} dW_{t}$
 
where $dS_t$ is the change in asset price, $\mu$ is the drift, $\sigma$ is the volatility, and $dW_t$ is the Wiener process.

Calculating Payoffs: For each simulated path, calculate the option's payoff based on its specific conditions (e.g., average price for Asian options, checking for barrier breaches for barrier options).

Averaging Payoffs: Calculate the average of all the simulated payoffs to get an estimate of the option's price.

Discounting: Discount the average payoff back to present value using the risk-free rate to get the option's fair price.


In [20]:
import numpy as np
import time
import matplotlib.pyplot as plt
import yfinance as yf
import datetime as dt
import pandas as pd
from dateutil.relativedelta import relativedelta

In [21]:
def simulate_stock_prices(S0, r, sigma, T, dt, paths):
    """Simulate stock prices using Geometric Brownian Motion."""
    
    steps = int(T / dt)
    dt = T / steps
    prices = np.zeros((steps + 1, paths))
    prices[0] = S0
    for t in range(1, steps + 1):
        z = np.random.standard_normal(paths)
        prices[t] = prices[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * z)
    return prices


In [22]:
def price_asian_option(S, K, r, T, option_type='call'):
    """Price an Asian option using Monte Carlo simulation."""

    avg_prices = np.mean(S, axis=0)
    if option_type == 'call': # Estimated option price
        payoffs = np.exp(-r * T) * np.maximum(avg_prices - K, 0)
    else:
        payoffs = np.exp(-r * T) * np.maximum(K - avg_prices, 0)
    return np.mean(payoffs)

def price_barrier_option(S, K, r, T, B, option_type='call', barrier_type='up-and-out'):
    """Price a Barrier option using Monte Carlo simulation."""

    if barrier_type == 'up-and-out':
        knocked_out = np.max(S, axis=0) > B
    else:  # 'down-and-out'
        knocked_out = np.min(S, axis=0) < B
    
    final_prices = S[-1]
    if option_type == 'call': # Estimated option price
        payoffs = np.where(knocked_out, 0, np.exp(-r * T) * np.maximum(final_prices - K, 0))
    else:
        payoffs = np.where(knocked_out, 0, np.exp(-r * T) * np.maximum(K - final_prices, 0))
    
    return np.mean(payoffs)


In [ ]:
# Define parameters for testing (Use cell below for specific ticker)
S0 = 100  # Initial stock price
K = 105  # Strike price
B = 110  # Barrier level
r = 0.055  # Risk-free rate
sigma = 0.2  # Volatility

In [23]:
ticker = "TSLA" # Ticker
r = 0.055 # Risk-free rate (US Treasury Interest Rate)

end_date = dt.date.today()
start_date = end_date - relativedelta(years=1)

ticker = yf.Ticker(ticker)
try:
    stock_data = ticker.history(start=start_date, end=end_date, auto_adjust=False, actions=False)
    stock_data = stock_data.sort_index(ascending=False)
except Exception as e:
    print(f"Error fetching stock data: {e}")
    stock_data = pd.DataFrame()

stock_data['Daily Returns'] = stock_data['Close'].pct_change() # Calculate daily returns

daily_volatility = stock_data['Daily Returns'].std() # Calculate daily volatility (standard deviation of daily returns)

sigma = daily_volatility * np.sqrt(252) # Annualised volatility (252 trading days)

S0 = stock_data['Close'].iloc[0] # Fetch latest close price for S0

K = 150 # Strike Price

B = 180 # Barrier level


In [24]:
T = 1  # Time to maturity (in years)
dt = 1/252  # Daily time step
paths_list = [50000, 100000, 500000, 1000000]  # Different numbers of paths to simulate

for paths in paths_list:
    start_time = time.time()
    S = simulate_stock_prices(S0, r, sigma, T, dt, paths)
    asian_price = price_asian_option(S, K, r, T)
    barrier_price = price_barrier_option(S, K, r, T, B, barrier_type="up-and-out")
    end_time = time.time()
    
    print(f"Paths: {paths}, Asian Option Price: {asian_price:.4f}, Barrier Option Price: {barrier_price:.4f}, Time: {end_time - start_time:.2f} seconds")


Paths: 50000, Asian Option Price: 36.1404, Barrier Option Price: 0.0537, Time: 0.56 seconds
Paths: 100000, Asian Option Price: 36.3389, Barrier Option Price: 0.0474, Time: 0.95 seconds
Paths: 500000, Asian Option Price: 36.0999, Barrier Option Price: 0.0488, Time: 6.21 seconds
Paths: 1000000, Asian Option Price: 36.1530, Barrier Option Price: 0.0494, Time: 15.05 seconds
